In [1]:
# 1. IMPORTS
import sys
import re
import os
from pathlib import Path
import pandas as pd

In [2]:
# 2. PATHS
PROJECT_ROOT = Path(r"C:\Users\dsgal\Documents\Finanças\personal_finance_app")
DATA_ABN_PATH = PROJECT_ROOT / "data" / "real" / "abn" / "abn_real.xls"
ACCOUNT_MAPPING_PATH = PROJECT_ROOT / "config" / "account_mapping.csv"

# Add project root to path and change working directory
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))
os.chdir(PROJECT_ROOT)

print(f"CWD: {Path.cwd()}")
print(f"ABN data exists? {DATA_ABN_PATH.exists()}")
print(f"Account mapping exists? {ACCOUNT_MAPPING_PATH.exists()}")

CWD: C:\Users\dsgal\Documents\Finanças\personal_finance_app
ABN data exists? True
Account mapping exists? True


In [3]:
# 3. ACCOUNT MAPPING
def load_account_mapping(path: Path = ACCOUNT_MAPPING_PATH) -> pd.DataFrame:
    """
    Load account number -> account name mapping from CSV.
    
    Expected columns: account_number, account_name
    Returns empty DataFrame if file doesn't exist (graceful fallback).
    """
    if not path.exists():
        print(f"⚠️  Account mapping not found: {path}")
        return pd.DataFrame(columns=["account_number", "account_name"])
    
    df = pd.read_csv(path, dtype={"account_number": str, "account_name": str})
    required_cols = {"account_number", "account_name"}
    missing = required_cols - set(df.columns)
    if missing:
        raise ValueError(f"Account mapping invalid. Missing columns: {missing}")
    
    # Normalize
    df["account_number"] = df["account_number"].astype(str).str.strip()
    df["account_name"] = df["account_name"].astype(str).str.strip()
    return df

In [4]:
# 4. LOAD AND CLEAN DATA
from src.data.abn.load_abn import load_abn

print("📥 Loading ABN data...")
df_raw = load_abn(DATA_ABN_PATH)
print(f"Loaded {len(df_raw)} transactions")

def apply_account_mapping(df: pd.DataFrame, account_map: pd.DataFrame) -> pd.DataFrame:
    """Replace account numbers with friendly names where available."""
    df = df.copy()
    df["accountNumber"] = df["accountNumber"].astype(str).str.strip()
    
    df = df.merge(account_map, how="left", left_on="accountNumber", right_on="account_number")
    df["accountNumber"] = df["account_name"].fillna(df["accountNumber"])
    return df.drop(columns=["account_number", "account_name"], errors="ignore")

def standardize_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    Rename columns to standard schema and drop unused columns.
    
    Standard schema:
    - Account: Account name/number
    - Currency: EUR/USD/etc
    - Date: datetime
    - Value: numeric amount (positive/negative)
    - Description: original text
    """
    df = df.copy()
    
    # Rename to standard names
    rename_map = {
        "accountNumber": "Account",
        "mutationcode": "Currency", 
        "transactiondate": "Date",
        "amount": "Value",
        "description": "Description"
    }
    df = df.rename(columns=rename_map)
    
    # Drop unused columns
    cols_to_drop = ["valuedate", "startsaldo", "endsaldo"]
    df = df.drop(columns=[c for c in cols_to_drop if c in df.columns], errors="ignore")
    
    # Type cleaning
    df["Account"] = df["Account"].astype(str).str.strip()
    df["Currency"] = df["Currency"].astype(str).str.strip().str.upper()
    df["Date"] = pd.to_datetime(df["Date"].astype(str), format="%Y%m%d", errors="coerce")
    df["Value"] = pd.to_numeric(df["Value"], errors="coerce")
    df["Description"] = df["Description"].astype(str)
    
    return df

# Apply account mapping and standardization
df_map = load_account_mapping()
df_clean = apply_account_mapping(df_raw, df_map)
df_clean = standardize_columns(df_clean)

print(f"✅ Clean data: {len(df_clean)} transactions")
print("Sample:")
print(df_clean[["Date", "Account", "Value", "Description"]].head())

📥 Loading ABN data...
Loaded 71 transactions
✅ Clean data: 71 transactions
Sample:
        Date Account    Value  \
0 2025-11-01     ABN   -29.00   
1 2025-11-03     ABN -1854.63   
2 2025-11-03     ABN    -7.27   
3 2025-11-04     ABN -7599.66   
4 2025-11-04     ABN  -500.00   

                                         Description  
0  BEA, Apple Pay                   Pulcinella En...  
1  /TRTP/SEPA Incasso algemeen doorlopend/CSID/NL...  
2  /TRTP/SEPA Incasso algemeen doorlopend/CSID/NL...  
3  /TRTP/SEPA Incasso algemeen doorlopend/CSID/NL...  
4  eCom, Apple Pay                  Revolut**4460...  


In [5]:
# 5. CATEGORIZATION
def clean_description_for_matching(description_series: pd.Series) -> pd.Series:
    """Clean description for rule matching (used in categorization)."""
    s = description_series.astype(str)
    s = s.str.replace(r"\s*\n\s*", " ", regex=True)  # Remove newlines
    s = s.str.replace(r"\s+", " ", regex=True)       # Collapse spaces
    s = s.str.strip().str.upper()
    return s

from src.utils.categorization import apply_categories

# Prepare data for categorization
df_categorization = pd.DataFrame({
    "date": df_clean["Date"],
    "description": clean_description_for_matching(df_clean["Description"]),
    "original_amount": df_clean["Value"],
    "original_currency": df_clean["Currency"],
    "account_source": df_clean["Account"],
    "institution": "ABN AMRO"
})

print("🏷️  Applying categories...")
df_final = apply_categories(df_categorization)
print("✅ Categorization complete")

🏷️  Applying categories...
Applying rule 0: pattern = 'PULCINELLA'
Applying rule 1: pattern = 'FUSION WOK'
Applying rule 2: pattern = 'AH '
Applying rule 3: pattern = 'AH TO GO'
Applying rule 4: pattern = 'JUMBO'
Applying rule 5: pattern = 'STHREE INTERIM SERVICES'
Applying rule 6: pattern = 'NIKE RETAIL'
Applying rule 7: pattern = 'TENNET'
Applying rule 8: pattern = 'HYPOTHEEK'
Applying rule 9: pattern = 'CENTRAAL BEHEER'
Applying rule 10: pattern = 'NS GROEP'
Applying rule 11: pattern = 'COINBASE IRELAND'
Applying rule 12: pattern = 'BELASTINGDIENST'
Applying rule 13: pattern = 'ABN AMRO SCHADEV'
Applying rule 14: pattern = 'SCILDON'
Applying rule 15: pattern = 'CZ GROEP'
Applying rule 16: pattern = 'DIER EN ZORG PLAN'
Applying rule 17: pattern = 'BIGGYM'
Applying rule 18: pattern = 'REVOLUT'
Applying rule 19: pattern = 'PAYPAL'
Applying rule 20: pattern = 'TENNET ARNHEM'
Applying rule 21: pattern = 'AAB INZ TIKKIE'
Applying rule 22: pattern = 'BUCKAROO'
Applying rule 23: pattern = '

In [6]:
# 6-9. SHORT DESCRIPTION TREATMENTS
def process_tikkie(description: str) -> str:
    """
    Tikkie treatment: Extract AUX (NAME) format.
    Example: "TIKKIE ID 123, KART, VAN JOHN DOE, IBAN..." → "KART (JOHN DOE)"
    """
    if not isinstance(description, str) or "TIKKIE ID" not in description.upper():
        return description
    
    # P1: After TIKKIE ID
    parts = description.split("TIKKIE ID", 1)
    if len(parts) < 2:
        return description
    after = parts[1].strip()
    
    # P2: After first ", "
    if ", " not in after:
        return description
    after = after.split(", ", 1)[1].strip()
    
    # P3: Until ", VAN"
    upper_after = after.upper()
    if ", VAN" not in upper_after:
        return description
    idx_van = upper_after.find(", VAN")
    aux = after[:idx_van].strip()
    
    # P4: Name after ", VAN" until next ", "
    resto = after[idx_van + 6:].strip()  # ", VAN" = 6 chars
    if ", " in resto:
        name = resto.split(", ", 1)[0].strip()
    else:
        name = resto.strip()
    
    return f"{aux} ({name})"

def process_apple_pay(description: str) -> str:
    """
    Apple Pay treatment: Remove prefixes and truncate at ",PAS".
    Example: "BEA, Apple Pay FUSION WOK,PAS..." → "FUSION WOK"
    """
    if not isinstance(description, str):
        return description
    
    t = description
    
    # Remove known prefixes
    prefixes = ["BEA, Apple Pay ", "eCom, Apple Pay ", "ECOM, Apple Pay ",
                "BEA, APPLE PAY ", "ECOM, APPLE PAY "]
    for prefix in prefixes:
        if t.startswith(prefix):
            t = t[len(prefix):]
            break
    
    # Truncate at ",PAS"
    if ",PAS" in t:
        t = t.split(",PAS", 1)[0]
    
    # Clean spaces
    t = re.sub(r"\s+", " ", t).strip()
    return t

def process_sepa(description: str) -> str:
    """
    SEPA treatment: Extract name after /NAME/ until next /.
    Example: "PAYMENT /NAME/JOHN DOE/MARF/..." → "JOHN DOE"
    """
    if not isinstance(description, str) or "/NAME/" not in description:
        return description
    
    # After /NAME/
    t = description.split("/NAME/", 1)[1]
    
    # Until next /
    if "/" in t:
        t = t.split("/", 1)[0]
    
    # Clean
    t = t.strip(" /").strip()
    t = re.sub(r"\s+", " ", t).strip()
    return t

def process_gea_betaalpas(description: str) -> str:
    """
    GEA BETAALPAS treatment: Extract store name + address.
    Example: "GEA, BETAALPAS GELDMAAT BURGEMEEST 88,PAS..." → "GELDMAAT BURGEMEEST 88"
    """
    if not isinstance(description, str) or not description.upper().startswith("GEA, BETAALPAS"):
        return description
    
    if ",PAS" in description:
        loja_endereco = description.split(",PAS", 1)[0].replace("GEA, BETAALPAS ", "")
        return loja_endereco.strip()
    return description

def process_basic_package(description: str) -> str:
    """
    ABN BASIC PACKAGE treatment.
    Example: "ABN AMRO BANK N.V. BASIC PACKAGE 3,70DEBIT..." → "BASIC PACKAGE"
    """
    if not isinstance(description, str) or "BASIC PACKAGE" not in description.upper():
        return description
    return "BASIC PACKAGE"

def process_revolut(description: str) -> str:
    """
    REVOLUT treatment: Simplify to just "REVOLUT".
    Example: "REVOLUT**4460* 19.11.25/09:44 DUBLIN..." → "REVOLUT"
    """
    if not isinstance(description, str) or not description.upper().startswith("REVOLUT"):
        return description
    return "REVOLUT"


def generate_short_descriptions(df: pd.DataFrame) -> pd.Series:
    """Apply all short description treatments in priority order."""
    descriptions = df["description"].astype(str).copy()
    
    # Priority 1: Tikkie
    mask_tikkie = descriptions.str.contains("TIKKIE ID", na=False, case=False)
    descriptions.loc[mask_tikkie] = descriptions.loc[mask_tikkie].apply(process_tikkie)
    
    # Priority 2: Apple Pay
    mask_apple = descriptions.str.contains("APPLE PAY", na=False, case=False)
    descriptions.loc[mask_apple] = descriptions.loc[mask_apple].apply(process_apple_pay)
    
    # Priority 3: SEPA
    mask_sepa = descriptions.str.contains("/NAME/", na=False)
    descriptions.loc[mask_sepa] = descriptions.loc[mask_sepa].apply(process_sepa)
    
    # Priority 4: GEA Betaalpas
    mask_gea = descriptions.str.contains("GEA, BETAALPAS", na=False, case=False)
    descriptions.loc[mask_gea] = descriptions.loc[mask_gea].apply(process_gea_betaalpas)
    
    # Priority 5: BASIC PACKAGE
    mask_package = descriptions.str.contains("BASIC PACKAGE", na=False, case=False)
    descriptions.loc[mask_package] = descriptions.loc[mask_package].apply(process_basic_package)
    
    # Priority 6: REVOLUT
    mask_revolut = descriptions.str.contains("REVOLUT", na=False, case=False)
    descriptions.loc[mask_revolut] = descriptions.loc[mask_revolut].apply(process_revolut)
    
    return descriptions

# Apply short description treatments
print("✂️  Generating short descriptions...")
df_final["short_description"] = generate_short_descriptions(df_final)

# Final result preview
print("\n🎉 FINAL RESULT (first 15 rows):")
preview_cols = ["date", "short_description", "original_amount", "transaction_type", "category", "subcategory"]
print(df_final[preview_cols].head(15))

# Show treatment examples
print("\n🔍 TREATMENT EXAMPLES:")
mask_tikkie = df_final["description"].str.contains("TIKKIE ID", na=False)
mask_apple = df_final["description"].str.contains("APPLE PAY", na=False, case=False)
mask_sepa = df_final["description"].str.contains("/NAME/", na=False)

print("Tikkie examples:")
print(df_final[mask_tikkie][["description", "short_description"]].head(2))
print("\nApple Pay examples:")
print(df_final[mask_apple][["description", "short_description"]].head(2))
print("\nSEPA examples:")
print(df_final[mask_sepa][["description", "short_description"]].head(2))

print(f"\n📊 SUMMARY: {len(df_final)} total transactions")
print(f"   Tikkie: {mask_tikkie.sum()}, Apple: {mask_apple.sum()}, SEPA: {mask_sepa.sum()}")

✂️  Generating short descriptions...

🎉 FINAL RESULT (first 15 rows):
         date                                 short_description  \
0  2025-11-01                               PULCINELLA ENSCHEDE   
1  2025-11-03                                  ABN AMRO BANK NV   
2  2025-11-03                                   CENTRAAL BEHEER   
3  2025-11-04                      AMERICAN EXPRESS EUROPE S.A.   
4  2025-11-04                                           REVOLUT   
5  2025-11-04                          COINBASE IRELAND LIMITED   
6  2025-11-05                          NS GROEP IZ NS REIZIGERS   
7  2025-11-06                              BCK*5831 AH AH TO GO   
8  2025-11-06                                     TENNET ARNHEM   
9  2025-11-06                                     TENNET ARNHEM   
10 2025-11-07                                           REVOLUT   
11 2025-11-07                        KART (M JORGE FALEIROS CJ)   
12 2025-11-07                         KART (R BONILHA FERRE

In [7]:
# 10. SAVE FINAL CSV
OUTPUT_PATH = PROJECT_ROOT / "data" / "processed" / "abn_transactions_final.csv"

# Create output directory if it doesn't exist
OUTPUT_PATH.parent.mkdir(parents=True, exist_ok=True)

# Select final columns for export (clean schema)
final_cols = [
    "date", "short_description", "description",  # descriptions (short + original)
    "original_amount", "original_currency", 
    "transaction_type", "category", "subcategory",
    "account_source", "institution", "Account", "Currency"
]

# Ensure all columns exist
export_cols = [col for col in final_cols if col in df_final.columns]

print(f"💾 Saving {len(df_final)} transactions to {OUTPUT_PATH}")
df_final[export_cols].to_csv(OUTPUT_PATH, index=False)

print(f"✅ Saved successfully: {OUTPUT_PATH}")
print(f"📁 File size: {OUTPUT_PATH.stat().st_size / 1024:.1f} KB")

# Quick validation
print("\n🔍 LAST 5 ROWS SAVED:")
print(df_final[export_cols].tail())


💾 Saving 71 transactions to C:\Users\dsgal\Documents\Finanças\personal_finance_app\data\processed\abn_transactions_final.csv
✅ Saved successfully: C:\Users\dsgal\Documents\Finanças\personal_finance_app\data\processed\abn_transactions_final.csv
📁 File size: 17.6 KB

🔍 LAST 5 ROWS SAVED:
         date                          short_description  \
66 2025-11-27  PRESENTE PARA VINI (JAIR DE SOUZA JUNIOR)   
67 2025-11-28                             BUDGET ENERGIE   
68 2025-11-28    PRESENTE PARA VINI (M DE SOUZA MARTINS)   
69 2025-11-28     ALMOCO FUSION WOK (E BASTOS BRUSCH CJ)   
70 2025-11-28    MW L C STEKKINGER VIA ING BETAALVERZOEK   

                                          description  original_amount  \
66  /TRTP/SEPA OVERBOEKING/IBAN/NL13ABNA0506417344...             40.0   
67  /TRTP/SEPA INCASSO ALGEMEEN DOORLOPEND/CSID/NL...           -180.0   
68  /TRTP/SEPA OVERBOEKING/IBAN/NL13ABNA0506417344...             10.0   
69  /TRTP/SEPA OVERBOEKING/IBAN/NL13ABNA0506417344...   

In [10]:
# 10. FINAL COLUMN RENAMING AND REORDERING (EXACT ORDER YOU WANTED)
print("🔄 Renaming and reordering columns to YOUR exact schema...")

# Step 1: Rename columns
rename_map = {
    "date": "DATE",
    "institution": "INSTITUTION", 
    "account_source": "ACCOUNT",
    "transaction_type": "TRANSACTION",
    "category": "CATEGORY",
    "subcategory": "SUBCATEGORY",
    "short_description": "DESCRIPTION",
    "original_amount": "AMOUNT",
    "original_currency": "CURRENCY",
    "description": "DETAILS"
}

df_final = df_final.rename(columns=rename_map)

# Step 2: YOUR EXACT column order
YOUR_EXACT_ORDER = [
    "DATE", "INSTITUTION", "ACCOUNT", "TRANSACTION", "CATEGORY", 
    "SUBCATEGORY", "DESCRIPTION", "AMOUNT", "CURRENCY", "DETAILS"
]

# Force YOUR exact column order
df_final = df_final.reindex(columns=YOUR_EXACT_ORDER)

print("✅ Final columns (YOUR order):", list(df_final.columns))
print("📊 Final shape:", df_final.shape)

# 11. SAVE FINAL CSV
OUTPUT_PATH = PROJECT_ROOT / "data" / "processed" / "abn_transactions_final.csv"
OUTPUT_PATH.parent.mkdir(parents=True, exist_ok=True)

print(f"💾 Saving {len(df_final)} transactions to {OUTPUT_PATH}")
df_final.to_csv(OUTPUT_PATH, index=False)

print(f"✅ Saved successfully: {OUTPUT_PATH}")
print(f"📁 File size: {OUTPUT_PATH.stat().st_size / 1024:.1f} KB")

# Final preview
print("\n🎉 FINAL PREVIEW (YOUR order):")
print(df_final.head().to_string())


🔄 Renaming and reordering columns to YOUR exact schema...
✅ Final columns (YOUR order): ['DATE', 'INSTITUTION', 'ACCOUNT', 'TRANSACTION', 'CATEGORY', 'SUBCATEGORY', 'DESCRIPTION', 'AMOUNT', 'CURRENCY', 'DETAILS']
📊 Final shape: (71, 10)
💾 Saving 71 transactions to C:\Users\dsgal\Documents\Finanças\personal_finance_app\data\processed\abn_transactions_final.csv
✅ Saved successfully: C:\Users\dsgal\Documents\Finanças\personal_finance_app\data\processed\abn_transactions_final.csv
📁 File size: 17.5 KB

🎉 FINAL PREVIEW (YOUR order):
        DATE INSTITUTION ACCOUNT TRANSACTION  CATEGORY          SUBCATEGORY                   DESCRIPTION   AMOUNT CURRENCY                                                                                                                                                                                                                                           DETAILS
0 2025-11-01    ABN AMRO     ABN     Expense   Leisure          Restaurants           PULCINELLA ENSC

In [11]:
df_final.head()

,DATE,INSTITUTION,ACCOUNT,TRANSACTION,CATEGORY,SUBCATEGORY,DESCRIPTION,AMOUNT,CURRENCY,DETAILS
0,2025-11-01,ABN AMRO,ABN,Expense,Leisure,Restaurants,PULCINELLA ENSCHEDE,-29.00,EUR,"BEA, APPLE PAY PULCINELLA ENSCHEDE,PAS021 NR:1..."
1,2025-11-03,ABN AMRO,ABN,Expense,Home,Mortgage,ABN AMRO BANK NV,-1854.63,EUR,/TRTP/SEPA INCASSO ALGEMEEN DOORLOPEND/CSID/NL...
2,2025-11-03,ABN AMRO,ABN,Expense,Taxes,Insurance,CENTRAAL BEHEER,-7.27,EUR,/TRTP/SEPA INCASSO ALGEMEEN DOORLOPEND/CSID/NL...
3,2025-11-04,ABN AMRO,ABN,Expense,Transfer,Credit card payment,AMERICAN EXPRESS EUROPE S.A.,-7599.66,EUR,/TRTP/SEPA INCASSO ALGEMEEN DOORLOPEND/CSID/NL...
4,2025-11-04,ABN AMRO,ABN,Expense,Transfer,Between accounts,REVOLUT,-500.00,EUR,"ECOM, APPLE PAY REVOLUT**4460* 04.11.25/11:57 ..."
